# Assignment 1

Deadline: 19.03.2025, 12:00 CET

<Add your name, student-id and emal address>

In [98]:
# Import standard libraries
import os
import sys
import timeit # To compute runtimes
from typing import Optional

# Import third-party libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import local modules
project_root = os.path.dirname(os.path.dirname(os.getcwd()))   # Change this path if needed
src_path = os.path.join(project_root, 'qpmwp-course', 'src')
sys.path.append(project_root)
sys.path.append(src_path)
print("current working directory:", os.getcwd())

from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.constraints import Constraints
from optimization.optimization import Optimization, Objective
from optimization.optimization_data import OptimizationData
from optimization.quadratic_program import QuadraticProgram, USABLE_SOLVERS

current working directory: c:\Users\Danin\Documents\Python Projects\Quant Portfolio UZH\qpmwp-course\assignments


## 1. Solver Horse Race

### 1.a)
(3 points)

Generate a Multivariate-Normal random dataset of dimension TxN, T=1000, N=100, and compute a vector of expected returns, q, and a covariance matrix, P, using classes ExpectedReturn and Covariance respectively.

In [104]:

# Set the dimensions
T = 1000  # Number of time periods
N = 100   # Number of assets
np.random.seed(42)

# Generate a random mean vector and covariance matrix for the multivariate normal distribution
mean = np.random.normal(0.001, 0.01, size=N)
cov = np.random.rand(N, N)
cov = np.dot(cov, cov.T)  

# Generate the Multivariate-Normal random dataset
data = np.random.multivariate_normal(mean, cov, T)

# Convert the dataset to a DataFrame for easier manipulation
df = pd.DataFrame(data, columns=[f'Asset_{i+1}' for i in range(N)])

# Compute the vector of expected returns (mean returns) from df
Expected_Return_Model = ExpectedReturn()
q = Expected_Return_Model.estimate(df, inplace = False)

# Compute the covariance matrix from df
covariance_model = Covariance()
P = covariance_model.estimate(df, inplace = False)

# Display the results
print("Vector of expected returns (q):")
print(q)

print("\nCovariance matrix (P):")
print(P)

Vector of expected returns (q):
Asset_1      2.032487
Asset_2      2.180768
Asset_3      2.344914
Asset_4      2.517583
Asset_5      2.120657
               ...   
Asset_96     2.022093
Asset_97     2.525798
Asset_98     2.123389
Asset_99     2.061985
Asset_100    2.240798
Length: 100, dtype: float64

Covariance matrix (P):
             Asset_1    Asset_2    Asset_3    Asset_4    Asset_5    Asset_6  \
Asset_1    28.399846  21.152870  21.787919  22.436343  18.891057  19.267133   
Asset_2    21.152870  34.311934  24.849765  25.893542  23.006202  23.542211   
Asset_3    21.787919  24.849765  32.869988  26.917292  20.624190  21.910006   
Asset_4    22.436343  25.893542  26.917292  37.555068  22.037301  23.782896   
Asset_5    18.891057  23.006202  20.624190  22.037301  28.965210  20.678069   
...              ...        ...        ...        ...        ...        ...   
Asset_96   20.381663  21.777796  22.139770  22.108893  19.224795  20.253742   
Asset_97   23.666869  26.443855  26.003148

c:\Users\Danin\Documents\Python Projects\Quant Portfolio UZH\venv\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


### 1.b)
(3 points)

Instantiate a constraints object by injecting column names of the data created in 1.a) as ids and add:
- a budget constaint (i.e., asset weights have to sum to one)
- lower bounds of 0.0 for all assets
- upper bounds of 0.2 for all assets
- group contraints such that the sum of the weights of the first 30 assets is <= 0.3, the sum of assets 31 to 60 is <= 0.4 and the sum of assets 61 to 100 is <= 0.5

In [100]:
# Instantiate the Constraints class
constraints = Constraints(ids = df.columns.tolist())

# Add budget constraint
constraints.add_budget(rhs=1, sense ='=')
print(constraints)
# Add box constraints (i.e., lower and upper bounds)
constraints.add_box(lower = 0.0, upper = 0.2)

# Add linear constraints
G = pd.DataFrame(np.zeros((3, N)), columns=constraints.ids)
G.iloc[0, 0:30] = 1
G.iloc[1, 30:60] = 1
G.iloc[2, 60:100] = 1
h = pd.Series([0.3, 0.4, 0.5])
constraints.add_linear(G=G, sense='<=', rhs=h)

constraints.budget
constraints.box
constraints.linear

{'G':    Asset_1  Asset_2  Asset_3  Asset_4  Asset_5  Asset_6  Asset_7  Asset_8  \
 0      1.0      1.0      1.0      1.0      1.0      1.0      1.0      1.0   
 1      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
 2      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
 
    Asset_9  Asset_10  ...  Asset_91  Asset_92  Asset_93  Asset_94  Asset_95  \
 0      1.0       1.0  ...       0.0       0.0       0.0       0.0       0.0   
 1      0.0       0.0  ...       0.0       0.0       0.0       0.0       0.0   
 2      0.0       0.0  ...       1.0       1.0       1.0       1.0       1.0   
 
    Asset_96  Asset_97  Asset_98  Asset_99  Asset_100  
 0       0.0       0.0       0.0       0.0        0.0  
 1       0.0       0.0       0.0       0.0        0.0  
 2       1.0       1.0       1.0       1.0        1.0  
 
 [3 rows x 100 columns],
 'sense': 0    <=
 dtype: object,
 'rhs': 0    0.3
 1    0.4
 2    0.5
 dtype: float64}

### 1.c) 
(4 points)

Solve a Mean-Variance optimization problem (using coefficients P and q in the objective function) which satisfies the above defined constraints.
Repeat the task for all open-source solvers in qpsolvers and compare the results in terms of:

- runtime
- accuracy: value of the primal problem.
- reliability: are all constarints fulfilled? Extract primal resisduals, dual residuals and duality gap.

Generate a DataFrame with the solvers as column names and the following row index: 'solution_found': bool, 'objective': float, 'primal_residual': float, 'dual_residual': float, 'duality_gap': float, 'runtime': float.

Put NA's for solvers that failed for some reason (e.g., unable to install the package or solvers throws an error during execution). 




In [101]:
# Extract the constraints in the format required by the solver
GhAb = constraints.to_GhAb()

outcome = pd.DataFrame(index=['solution_found', 'objective', 'primal_residual', 'dual_residual', 'duality_gap', 'runtime'])

risk_aversion = 3

# Loop over solvers, instantiate the quadratic program, solve it and store the results
for solver in USABLE_SOLVERS:
    start_time = timeit.default_timer()
    
    qp = QuadraticProgram(
        P=P.to_numpy() * risk_aversion,
        q=q.to_numpy() * -1,
        G=GhAb['G'],
        h=GhAb['h'],
        A=GhAb['A'],
        b=GhAb['b'],
        lb=constraints.box['lower'].to_numpy(),
        ub=constraints.box['upper'].to_numpy(),
        solver=solver,
    ) 

    qp.solve()
    
    runtime = timeit.default_timer() - start_time
    
    solution = qp.results.get('solution')
    
    outcome.loc[:, solver] = [
        solution.found, 
        qp.objective_value(), 
        solution.primal_residual(), 
        solution.dual_residual(), 
        solution.duality_gap()[0], 
        runtime
    ]
print(outcome)
print(solution)


                     highs      osqp       daqp   quadprog     cvxopt  \
solution_found        True      True       True       True       True   
objective        26.324539  26.03891  26.324539  26.324539  26.324539   
primal_residual        0.0  0.001196        0.0        0.0        0.0   
dual_residual          0.0  0.004301        0.0        0.0        0.0   
duality_gap            0.0  0.279524        0.0        0.0   0.000001   
runtime             0.0108   0.00681   0.001765   0.006291   0.039788   

                     qpalm  
solution_found        True  
objective        26.322067  
primal_residual   0.000033  
dual_residual          0.0  
duality_gap       0.002471  
runtime           0.008589  
Solution(problem=<qpsolvers.problem.Problem object at 0x0000026BFEB9D890>, extras={'info': <qpalm._qpalm.Info object at 0x0000026BFEBADF70>}, found=True, obj=None, x=array([-3.20484728e-06, -2.00219528e-06, -2.54270934e-06, -1.88767248e-06,
       -1.14513575e-05, -6.03879509e-06, -1.

Print and visualize the results

In [102]:
outcome

,highs,osqp,daqp,quadprog,cvxopt,qpalm
solution_found,True,True,True,True,True,True
objective,26.324539,26.03891,26.324539,26.324539,26.324539,26.322067
primal_residual,0.0,0.001196,0.0,0.0,0.0,0.000033
dual_residual,0.0,0.004301,0.0,0.0,0.0,0.0
duality_gap,0.0,0.279524,0.0,0.0,0.000001,0.002471
runtime,0.0108,0.00681,0.001765,0.006291,0.039788,0.008589


## 2. Analytical Solution to Minimum-Variance Problem

(5 points)

- Create a `MinVariance` class that follows the structure of the `MeanVariance` class.
- Implement the `solve` method in `MinVariance` such that if `solver_name = 'analytical'`, the analytical solution is computed and stored within the object (if such a solution exists). If not, call the `solve` method from the parent class.
- Create a `Constraints` object by injecting the same ids as in part 1.b) and add a budget constraint.
- Instantiate a `MinVariance` object by setting `solver_name = 'analytical'` and passing instances of `Constraints` and `Covariance` as arguments.
- Create an `OptimizationData` object that contains an element `return_series`, which consists of the synthetic data generated in part 1.a).
- Solve the optimization problem using the created `MinVariance` object and compare the results to those obtained in part 1.c).


In [103]:
# Define class MinVariance
class MinVariance(Optimization):

    def __init__(self,
                 constraints: Constraints,
                 covariance: Optional[Covariance] = None,
                 **kwargs):
        super().__init__(
            constraints=constraints,
            **kwargs
        )
        self.covariance = Covariance() if covariance is None else covariance

    def set_objective(self, optimization_data: OptimizationData) -> None:
        X = optimization_data['return_series']
        covmat = self.covariance.estimate(X=X, inplace=False)
        self.objective = Objective(
            P = covmat * 2, 
            q = np.zeros(covmat.shape[0])
        )

    def solve(self) -> None:
        if self.params.get('solver_name') == 'analytical':
            ones = np.ones(P.shape[0])
            
            try:
                inv_P = np.linalg.inv(P)
                weights = inv_P @ ones / (ones.T @ inv_P @ ones)
                self.results = {'weights': weights}
                self.solution_found = True
            except np.linalg.LinAlgError:
                self.solution_found = False
                self.results = {'weights': None}
            return None
        else:
            return super().solve()


# Create a constraints object with just a budget constraint
constraints = Constraints(ids = df.columns.tolist())
constraints.add_budget(rhs=1, sense ='=')

# Instantiate the MinVariance class
min_var = MinVariance(solver_name='analytical', constraints=constraints, covariance=covariance_model)

# Prepare the optimization data and prepare the optimization problem
optimization_data = OptimizationData(return_series=df)
min_var.set_objective(optimization_data)

# Solve the optimization problem and print the weights
min_var.solve()
print("Optimal Weights:", min_var.results['weights'])

min_var2 = MinVariance(constraints=constraints, covariance=covariance_model)

# Prepare the optimization data and prepare the optimization problem
optimization_data = OptimizationData(return_series=df)
min_var2.set_objective(optimization_data)

# Solve the optimization problem and print the weights
min_var2.solve()
print("Optimal Weights:", min_var2.results['weights'])

Optimal Weights: [-3.11969504 -2.48845201  1.47155387  4.36172684  5.67026669 -2.79044655
 -1.3314308   1.81719326 -1.55010998 -2.1106496   1.16748605  4.9885363
  1.55652297  1.058947   -4.45807982  3.64767407  2.42153965 -8.49693749
  6.59751367 -0.50255251  3.13339525  4.16228096 -0.66655267 -3.87659568
 -8.00924233 -2.76527683  0.97359394 -8.78583536  7.53919776 -7.90247535
 -3.23433649  6.53744989  6.99917184 -1.61166108  5.13922983  2.63159214
  4.36480137  1.16209321  6.23899856 -2.31035377 -2.79134896 -2.72300326
 -1.6500431  -1.38563386  0.74419358  2.07013539 -8.2482382  -0.62598184
 -4.16028021  3.19598819  6.44405922 -5.31283822 -6.58679096  4.86551222
  4.36201617 -3.44638262  3.40127479 -4.67118891  3.32300654  5.71416654
 -2.28845046 -0.69212219 -6.00482572  0.73830593 -3.55055963 -2.75103281
  0.87266086 -2.99872573 -6.88140764  4.35108565  0.12665684  0.20247669
  4.28794875 -0.14452701  3.43873555  2.32470049 -0.56617562  1.66492358
 -3.33447331  2.15273554  5.7708924